# <span style="color:darkblue">04-Spot_Assignment</span>

In this notebook we will assign the detected RNAs and Txs to each uniquely labeleld cell in our Cellpose-produced cell and nuclear masks.

## 4.0 - Load libraries

In [28]:
pwd

'G:\\Code\\smFISH\\notebooks'

In [1]:
from skimage import io
import numpy as np
import napari
import pandas as pd
from glob import glob
from skimage.measure import regionprops_table, regionprops

import sys
sys.path.append('../')

from src.misc import group_experiments, load_data

C:\Users\roana\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


***

## 4.1 - Load spot & cell/nuclei mask data and check image

In [3]:
# load image data
RNAs = io.imread('../data/Chip_A_spider_1_CLB2_HWP1_24HR_RESULTSDONE/CET111_HWP1Q610_Spider_CY3_01.tif')
DAPI = io.imread('../data/Chip_A_spider_1_CLB2_HWP1_24HR_RESULTSDONE/stk_0003_CET111_EFG1Q670_HWP1CAL610_Spider37_01_CY5, CY3.5 NAR, DAPI.tif')
DIC = io.imread('../data/Chip_A_spider_1_CLB2_HWP1_24HR_RESULTSDONE/CET111_EFG1Q670_HWP1CAL610_Spider37_01_BF.tif')
mask = io.imread('../data/Chip_A_spider_1_CLB2_HWP1_24HR_RESULTSDONE/Masks/CET111_HWP1Q610_Spider_DIC_01_seg.tif')
nuclear_mask = io.imread('../data/Chip_A_spider_1_CLB2_HWP1_24HR_RESULTSDONE/Masks/MAX_CET111_HWP1Q610_Spider_DAPI_01_seg.tif')

# load spot data
spot_data = np.load(glob('../data/Chip_A_spider_1_CLB2_HWP1_24HR_RESULTSDONE/Spots/CET111_HWP1Q610_Spider_CY3_01_spots_thr5.npy')[0])
dense_data = np.load(glob('../data/*/Spots decomposition/CET111_HWP1Q610_Spider_CY3_01_spots_thr5_dd_regions.npy')[0])

FileNotFoundError: [Errno 2] No such file or directory: 'F:\\Code\\smFISH\\data\\Chip_A_spider_1_CLB2_HWP1_24HR_RESULTSDONE\\stk_0003_CET111_EFG1Q670_HWP1CAL610_Spider37_01_CY5, CY3.5 NAR, DAPI.tif'

In [4]:
def preprocess_spot_data(spot_data, dense_data):
    # spot_data has the form:
    # z, y, x

    # dense data has the form 
    # z, y, x, mRNA counts, -- other information --

    # let's introduce mRNA counts of 1 for the spots:    
    spot_data_padded = np.pad(spot_data, ((0,0),(0,1)), mode='constant', constant_values=1)
    
    # discard other information and merge
    spot_data_combined = np.concatenate([spot_data_padded, dense_data[:,:4]], axis=0)
    return spot_data_combined


def count_spots(mask, nuclear_mask, spot_data, cells):    
    for z, y, x, number in spot_data:
        cell_id = mask[y, x]
        nucleus = nuclear_mask[y, x]

        if number == 1:
            cells[cell_id]['spots_per_cell'] += number
        else:
            cells[cell_id]['dense_regions_per_cell'] += 1
            cells[cell_id]['decomposed_RNAs'] += number

            # if the spot sits in the nucleus, 
            # also increase nascent RNAs and transcription sites
            if nucleus > 0:
                cells[cell_id]['tx_per_cell'] += 1
                cells[cell_id]['nascent_RNAs'] += number
    return cells

def count_nuclei(mask, nuclear_mask, cells):
    # count nuclei per cell - hyphae may have multiple ones!
    for nucleus in regionprops(nuclear_mask):
        y, x = nucleus.centroid
        cell_id = mask[int(y), int(x)]
        cells[cell_id]['nuclei'] += 1
    return cells

def spot_assignment(mask, nuclear_mask, spot_data, dense_data):
    cells = {}
    
    for cell_id in np.unique(mask):
        cells[cell_id] = {
            'spots_per_cell': 0,
            'dense_regions_per_cell': 0,
            'decomposed_RNAs': 0,
            'tx_per_cell': 0,
            'nascent_RNAs': 0,
            'nuclei': 0
        }
        
    spot_data_combined = preprocess_spot_data(spot_data, dense_data)
    
    cells = count_spots(mask, nuclear_mask, spot_data_combined, cells)
    cells = count_nuclei(mask, nuclear_mask, cells)
    
    # remove spots on background
    del cells[0]

    # convert to dataframe, collect object information and merge
    df = pd.DataFrame(cells).T.reset_index().rename(columns={'index': 'label'})
    df['total_RNAs_per_cell'] = df['spots_per_cell'] + df['decomposed_RNAs'] - df['dense_regions_per_cell']

    props = pd.DataFrame(regionprops_table(mask.astype(int), properties=['label', 'bbox', 'area', 'eccentricity', 'major_axis_length']))
    df = props.merge(df, on='label')

    return df

In [4]:
df = spot_assignment(mask, nuclear_mask, spot_data, dense_data)

In [5]:
df.head()

,label,bbox-0,bbox-1,bbox-2,bbox-3,area,eccentricity,spots_per_cell,dense_regions_per_cell,decomposed_RNAs,tx_per_cell,nascent_RNAs,nuclei,total_RNAs_per_cell
0,1,0,425,27,486,1364.0,0.908397,0,0,0,0,0,1,0
1,2,0,367,571,471,12625.0,0.986942,8,1,3,0,0,3,10
2,3,215,0,533,670,16808.0,0.918540,0,0,0,0,0,1,0
3,4,329,666,364,714,1327.0,0.673452,0,0,0,0,0,1,0
4,5,305,560,352,631,2503.0,0.787041,1,0,0,0,0,1,1


***

In [9]:
#ADDED MYSELF FROM CODE BELOW
savename = "TESTING"
root_dir = '../data/Chip_7'


savename = f"{root_dir}/Results/TESTING.csv"


df.to_csv(savename)

## 4.2 - Batch assignment

In [11]:
root_dir = '../data/Chip_B_Spider_1_EFG1_HWP1_24HR_RESULTSDONE'

In [12]:
experiments = group_experiments(root_dir)

print('I found the following experiments:')
print(experiments.keys())
print('select applicable experiments')

I found the following experiments:
dict_keys(['CET111_EFG1Q670_Spider', 'CET111_HWP1Q610_Spider'])
select applicable experiments


In [14]:
experiments_to_process = ['CET111_EFG1Q670_Spider']

for identifier in experiments_to_process:
    print(identifier)
    replicates = experiments[identifier]
    
    for replicate, paths, in replicates.items():

        print(f'processing {identifier=}, {replicate=}')
        data = load_data(paths)
        savename = f"{root_dir}/Results/{paths['output_name']}.csv"

        process = True
        # check if all files required for this step have been loaded
        for entry in ['spots', 'dense', 'cell_mask', 'nuclear_mask']:
            if data.get(entry) is None:
                print(f'{identifier=}, {replicate=}, {entry=} could not be found')
                print(f'skipping {identifier=}, {replicate=}!')
                process=False

        if process:
            df = spot_assignment(
                data.get('cell_mask'), 
                data.get('nuclear_mask'),
                data.get('spots'),
                data.get('dense')
            )

            print(f'saving data to {savename}')
            df.to_csv(savename)
            print('done.')

        print(10*'-')

CET111_EFG1Q670_Spider
processing identifier='CET111_EFG1Q670_Spider', replicate=1


DeflateError: libdeflate_zlib_decompress returned LIBDEFLATE_BAD_DATA